In [2]:
import optuna
import os

import pandas as pd
from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_columns = None
pd.set_option("display.max_columns", None)
pd.set_option("display.max_columns", None)  # or 1000
pd.set_option("display.max_rows", None)  # or 1000
pd.set_option("display.max_colwidth", -1)  # or 199

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

from warnings import simplefilter

simplefilter(action="ignore", category=DeprecationWarning)

/tmp/ipykernel_100122/712628210.py:13: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)  # or 199


In [4]:
pre = f"copy_tune_2"
pre = f"copy_adap_1"

env = "bmfl"
alpha = 0.1
dataset = "cifar10"

study_name = f"env_{env}_seed_777_d_{dataset}_alpha_{alpha}_gdt_2"


def f():
    print(f"[+] pre: {pre}")
    print(f"[+] Study: {study_name}")

    storage = f"sqlite:///studys/{pre}/{study_name}.db"
    study = optuna.load_study(study_name=study_name, storage=storage)

    def constraints_func(trial):
        try:
            if trial.params["num_gibbs_draws_train"]:
                return [0]
            if trial.params["num_gibbs_draws_test"]:
                return [0]
        except:
            pass
        return [1]

    # constraints_func = None
    display(optuna.visualization.plot_pareto_front(study, target_names=["best_test_acc", "best_ood_acc"], constraints_func=constraints_func))

    df = study.trials_dataframe()
    df.drop(columns=["datetime_start", "datetime_complete", "duration"], inplace=True)
    df["values_sum"] = df["values_0"] + df["values_1"]
    # df[df.columns].sort_values(by=['values_sum','values_0','values_1'], ascending=[False,False,False]).head(10)
    # df[df.columns].sort_values(by=['values_0','values_1'], ascending=[False,False]).drop_duplicates(subset=["values_0", "values_1"], keep='last').head(10)
    # df[df.columns].sort_values(by=['values_1','values_0'], ascending=[False,False]).drop_duplicates(subset=["values_0", "values_1"], keep='first').head(10)
    df_filtered_test = df[df.columns].sort_values(by=["values_0", "values_1"], ascending=[False, False]).drop_duplicates(subset=["values_0", "values_1"], keep="first")
    df_filtered_ood = df[df.columns].sort_values(by=["values_1", "values_0"], ascending=[False, False]).drop_duplicates(subset=["values_0", "values_1"], keep="first")
    print(f"[+] total trials: {df_filtered_test.shape[0]}")
    display(df_filtered_test.head(6))
    display(df_filtered_ood.head(6))
    display(df[df.columns].sort_values(by=["values_1", "values_0"]))


f()

[+] pre: copy_adap_1
[+] Study: env_bmfl_seed_777_d_cifar10_alpha_0.1_gdt_2


/gallery_moma/junhyeog.yun/anaconda3/envs/gp-my/lib/python3.9/site-packages/optuna/visualization/_pareto_front.py:227: ExperimentalWarning:

``constraints_func`` argument is an experimental feature. The interface can change in the future.



[+] total trials: 10


,number,values_0,values_1,params_adap_epochs,params_inner_steps,params_lr,params_num_gibbs_draws_test,params_num_gibbs_draws_train,params_wd,state,values_sum
14,14,0.328533,0.277103,1.0,3.0,0.065,30.0,20.0,0.0010,COMPLETE,0.605636
7,7,0.312074,0.251012,NaN,3.0,0.030,30.0,20.0,0.0010,COMPLETE,0.563086
12,12,0.274913,0.312641,5.0,5.0,0.025,30.0,20.0,0.0006,COMPLETE,0.587554
11,11,0.250482,0.147548,3.0,3.0,0.775,30.0,20.0,0.0006,COMPLETE,0.398031
5,5,0.234281,0.225821,NaN,1.0,0.030,30.0,20.0,0.0000,COMPLETE,0.460102
13,13,0.226566,0.170490,5.0,3.0,0.345,30.0,20.0,0.0009,COMPLETE,0.397056


,number,values_0,values_1,params_adap_epochs,params_inner_steps,params_lr,params_num_gibbs_draws_test,params_num_gibbs_draws_train,params_wd,state,values_sum
12,12,0.274913,0.312641,5.0,5.0,0.025,30.0,20.0,0.0006,COMPLETE,0.587554
14,14,0.328533,0.277103,1.0,3.0,0.065,30.0,20.0,0.0010,COMPLETE,0.605636
7,7,0.312074,0.251012,NaN,3.0,0.030,30.0,20.0,0.0010,COMPLETE,0.563086
5,5,0.234281,0.225821,NaN,1.0,0.030,30.0,20.0,0.0000,COMPLETE,0.460102
6,6,0.223094,0.201529,NaN,1.0,0.030,30.0,20.0,0.0006,COMPLETE,0.424623
9,9,0.190690,0.183086,3.0,1.0,0.995,30.0,20.0,0.0009,COMPLETE,0.373776


,number,values_0,values_1,params_adap_epochs,params_inner_steps,params_lr,params_num_gibbs_draws_test,params_num_gibbs_draws_train,params_wd,state,values_sum
10,10,0.154430,0.114260,5.0,1.0,0.545,30.0,20.0,0.0007,COMPLETE,0.268690
11,11,0.250482,0.147548,3.0,3.0,0.775,30.0,20.0,0.0006,COMPLETE,0.398031
13,13,0.226566,0.170490,5.0,3.0,0.345,30.0,20.0,0.0009,COMPLETE,0.397056
9,9,0.190690,0.183086,3.0,1.0,0.995,30.0,20.0,0.0009,COMPLETE,0.373776
6,6,0.223094,0.201529,NaN,1.0,0.030,30.0,20.0,0.0006,COMPLETE,0.424623
5,5,0.234281,0.225821,NaN,1.0,0.030,30.0,20.0,0.0000,COMPLETE,0.460102
7,7,0.312074,0.251012,NaN,3.0,0.030,30.0,20.0,0.0010,COMPLETE,0.563086
14,14,0.328533,0.277103,1.0,3.0,0.065,30.0,20.0,0.0010,COMPLETE,0.605636
12,12,0.274913,0.312641,5.0,5.0,0.025,30.0,20.0,0.0006,COMPLETE,0.587554
0,0,NaN,NaN,NaN,5.0,0.030,30.0,20.0,0.0006,FAIL,NaN


In [5]:
storage = f"sqlite:///studys/{pre}/{study_name}.db"
study = optuna.load_study(study_name=study_name, storage=storage)

In [6]:
optuna.visualization.plot_param_importances(study, target=lambda t: t.values[0])
optuna.visualization.plot_param_importances(study, target=lambda t: t.values[1])

/gallery_moma/junhyeog.yun/anaconda3/envs/gp-my/lib/python3.9/site-packages/optuna/visualization/_utils.py:69: UserWarning:

`target` is specified, but `target_name` is the default value, 'Objective Value'.



/gallery_moma/junhyeog.yun/anaconda3/envs/gp-my/lib/python3.9/site-packages/optuna/visualization/_utils.py:69: UserWarning:

`target` is specified, but `target_name` is the default value, 'Objective Value'.



In [7]:
optuna.visualization.plot_contour(study, target=lambda t: sum(t.values)).show("browser")

/gallery_moma/junhyeog.yun/anaconda3/envs/gp-my/lib/python3.9/site-packages/optuna/visualization/_utils.py:69: UserWarning:

`target` is specified, but `target_name` is the default value, 'Objective Value'.

[W 2023-08-26 09:02:46,210] Param num_gibbs_draws_test unique value length is less than 2.
[W 2023-08-26 09:02:46,211] Param num_gibbs_draws_train unique value length is less than 2.
[W 2023-08-26 09:02:46,213] Param num_gibbs_draws_test unique value length is less than 2.
[W 2023-08-26 09:02:46,214] Param num_gibbs_draws_train unique value length is less than 2.
[W 2023-08-26 09:02:46,215] Param num_gibbs_draws_test unique value length is less than 2.
[W 2023-08-26 09:02:46,217] Param num_gibbs_draws_test unique value length is less than 2.
[W 2023-08-26 09:02:46,218] Param num_gibbs_draws_train unique value length is less than 2.
[W 2023-08-26 09:02:46,218] Param num_gibbs_draws_test unique value length is less than 2.
[W 2023-08-26 09:02:46,219] Param num_gibbs_draws_train uniqu